### Datasets

We need to download some data for the examples. Some files are bundled with the notebooks because I am not smart enough to understand how to download them through the various web / ftp interfaces directly or I needed to filter or compress the information for the purposes of the class. Please go to the original sites if you need to use these data for anything other than the demonstrations in these notebooks.

Most datasets are too large for the repository (and generally, that's not a place to keep anything which is not the primary target of revision management. The bundled data are compressed and will be unpacked (copied) to the `Resources` directory by the "download" functions in this notebook. If you mess them up, just run the download again. Anything that is undamaged will just be be skipped anyway.

#### Global Magnetic Data

Magnetic intensity data from [geomag.org](http://geomag.org/models/EMAG2/EMAG2_V2.tif)

#### Topography data

ETOPO1 images are from NOAA - we use their geotifs which are subsampled from the original (enormous) dataset but 

#### NASA Blue marble images

Winter and Summer images for the Earth are grabbed for plotting examples. The winter ones (June) are used by default as these have less ice in the N. Hemisphere. 

#### Earthquake hypocentres

Are grabbed from the [NEIC](http://earthquake.usgs.gov/earthquakes/search/) - they are in the geoJSON format since that is very simple to read with python. The downloads are limited at 20k events so the time and magnitude range is whatever it takes to get just under this limit. The filenames give clues about that, but, so does the catalogue itself once it is read in.

#### Global age grid 

Taken from Earthbyte and reduced in size by throwing away the grid information and saving in compressed numpy format. 

#### Global strain rate

I grabbed the second invariant of the strain rate from the [global strain rate map](http://gsrm.unavco.org/intro) project through the 'Jules Vernes' portal. 



In [13]:
## Here are some functions to download or install files. 


import requests
import os
import time
import sys
import shutil

def download_file(url, local_filename, expected_size):
    
    
# We might want to bundle some files if they are small / compressed or not readily available for download

    if (os.path.isfile(url)):
        shutil.copy(url, local_filename)
    
    else:
        r = requests.get(url, stream=True)

        start_time = time.time()
        last_time = start_time
        datasize = 0

        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=10000000): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()
                    datasize += len(chunk)
                    if (time.time() - last_time) > 2.5:
                        print "{:.1f} Mb in {:.1f}s / {}".format(datasize / 1.0e6, time.time() - start_time, expected_size)
                        last_time = time.time()

    return 

import hashlib

def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "r+b") as f:
        for block in iter(lambda: f.read(blocksize), ""):
            hash.update(block)
    return hash.hexdigest()


def download_cached_file(location_url, local_file, expected_md5, expected_size="Unknown"):

    try:
        assert md5sum(local_file) == expected_md5
        print "Using cached file - {}".format(local_file)
        return(2)
    
    except (IOError, AssertionError) as error_info:
        # No file or the wrong file ... best go download it
        # print "Assertion failed - {}".format(sys.exc_info())
        
        try:
            data_file = download_file(location_url, local_file, expected_size)
            print "Downloaded from {}".format( location_url )
            return(1)

        except:
            print "Unable to download {} [{}] ".format( location_url, sys.exc_info() )
            return(0)


def report_cached_file(local_file, expected_md5):
    if len(expected_md5) == 0 or md5sum(local_file) == expected_md5:
        print "Cached file {} is valid".format(local_file)
    else:
        print "Cached file {} failed, checksum {}".format(local_file, md5sum(local_file))
        
        

In [2]:
# Resources required for the tutorial
# The checksum is to make sure any cached file is the right one.
# If the checksum is wrong the file gets downloaded again.

# Some of these files are local, stored in the repo in compressed form and installed into the
# Resources directory. 

resource_list = [
# Global magnetic data 
{
 "local_file":"Resources/EMAG2_image_V2.tif",
 "md5":"20024c06fdabfa57e4f9efb4aabec198",
 "url":"http://geomag.org/models/EMAG2/EMAG2_image_V2.tif",
 "expected_size":"133Mb"
},
    
# Australian Mag Data from AuScope portal (built by hand)
{
 "local_file":"Resources/AusMagAll.tiff.zip",
 "md5":'',
 "url":"Reference/AusMagAll.tiff.zip",
 "expected_size":"10Mb"
},
    
# Blue Marble Image Geotiff June 2004 (11Mb)  
{
 "local_file": "Resources/BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF",
 "md5": '55e1399674d43a26353d84c114d7ff80',
 "url":"http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526294&cs=rgb&format=TIFF&width=3600&height=1800",
 "expected_size":"11Mb"
},
    
    
# Blue Marble Image with Bathymetry png - December 2004 
{
 "local_file": "Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF",
 "md5": '825da4b417ae19e24d2ea6db6cf8ad21',
 "url":"http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526311&cs=rgb&format=TIFF&width=3600&height=1800",
 "expected_size":"11Mb"
},
    
# Etopo1 - color image 
{
 "local_file": "Resources/color_etopo1_ice_low.tif.zip",
 "md5": '3f53d72e85393deb28874db0c76fbfcb',
 "url":"https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/image/color_etopo1_ice_low.tif.zip",
 "expected_size":"35Mb"
},
    
# Etopo1 - bw shaded relief image 
{
 "local_file": "Resources/etopo1_grayscale_hillshade.tif.zip",
 "md5": 'f38b8dc6cd971d72e77edaa837d5b85c',
 "url":"https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/image/etopo1_grayscale_hillshade.tif.zip",
 "expected_size":"115Mb"
},
    
    
# ETOPO_c_geotiff_10800 ... cell centred data for ETOPO1 
{
 "local_file": "Resources/ETOPO_c_geotiff_10800.tif.zip",
 "md5":"fbd0478d0974ca433cdd5cb3530d0d48",
 "url":"https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/data/ice_surface/cell_registered/georeferenced_tiff/ETOPO1_Ice_c_geotiff.zip",
 "expected_size":"320Mb"
},    
    
# Natural Earth Hypsometry Images (same grid size as ETOPO1 image above)
{
 "local_file":"Resources/HYP_50M_SR_W.zip",
 "md5":"f3323bc6f38ddec98cff4936fb324ec5",
 "url":"http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/raster/HYP_50M_SR_W.zip",
 "expected_size":"102Mb"
},   
 
# Natural Earth Ocean Bottom Images (same grid size as ETOPO1 image above)
{
 "local_file":"Resources/OB_50M.zip",
 "md5":"69a2ed6ff8cf30b2dedd2d80461f9e43",
 "url":"http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/raster/OB_50M.zip",
 "expected_size":"50Mb"
},   
            
# Global age grid  [LOCAL]
{
 "local_file":"Resources/global_age_data.3.6.z.npz",
 "md5":'',
 "url":"Reference/global_age_data.3.6.z.npz",
 "expected_size":"22Mb"
},
      
# Global second invariant of strain rate [LOCAL]
{
 "local_file":"Resources/sec_invariant_strain_0.2.dat.zip",
 "md5":'',
 "url":"Reference/sec_invariant_strain_0.2.dat.zip",
 "expected_size":"4Mb"
},
  
    
# Earthquake data for various regions as json files [LOCAL]
{
 "local_file":"Resources/Earthquake_data.zip",
 "md5":'',
 "url":"Reference/Earthquake_data.zip",
 "expected_size":"4Mb"
},
    
    
# Global surface velocity vector fields in various reference frames:
# http://gsrm.unavco.org/model/
# There are others ... 
    
{
 "local_file":"Resources/velocity_AU.nc",
 "md5":'',
 "url":"Reference/velocity_AU.nc",
 "expected_size":"1Mb"
},   
  
{
 "local_file":"Resources/velocity_EU.nc",
 "md5":'',
 "url":"Reference/velocity_EU.nc",
 "expected_size":"1Mb"
},   


{
 "local_file":"Resources/velocity_IN.nc",
 "md5":'',
 "url":"Reference/velocity_IN.nc",
 "expected_size":"1Mb"
},   


{
 "local_file":"Resources/velocity_NA.nc",
 "md5":'',
 "url":"Reference/velocity_NA.nc",
 "expected_size":"1Mb"
},   


{
 "local_file":"Resources/velocity_OK.nc",
 "md5":'',
 "url":"Reference/velocity_OK.nc",
 "expected_size":"1Mb"
},   


{
 "local_file":"Resources/velocity_NNR.nc",
 "md5":'',
 "url":"Reference/velocity_NNR.nc",
 "expected_size":"1Mb"
},   


{
 "local_file":"Resources/velocity_PA.nc",
 "md5":'',
 "url":"Reference/velocity_PA.nc",
 "expected_size":"1Mb"
},   


{
 "local_file":"Resources/velocity_TA.nc",
 "md5":'',
 "url":"Reference/velocity_TA.nc",
 "expected_size":"1Mb"
},   


       
]



In [14]:
for resource in resource_list:
    print "\nChecking {:s} ({})".format(resource["local_file"], resource["md5"])
    report_cached_file(resource["local_file"],resource["md5"])


Checking Resources/EMAG2_image_V2.tif (20024c06fdabfa57e4f9efb4aabec198)
Cached file Resources/EMAG2_image_V2.tif is valid

Checking Resources/AusMagAll.tiff.zip ()
Cached file Resources/AusMagAll.tiff.zip is valid

Checking Resources/BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF (55e1399674d43a26353d84c114d7ff80)
Cached file Resources/BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF is valid

Checking Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF (825da4b417ae19e24d2ea6db6cf8ad21)
Cached file Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF is valid

Checking Resources/color_etopo1_ice_low.tif.zip (3f53d72e85393deb28874db0c76fbfcb)
Cached file Resources/color_etopo1_ice_low.tif.zip is valid

Checking Resources/etopo1_grayscale_hillshade.tif.zip (f38b8dc6cd971d72e77edaa837d5b85c)
Cached file Resources/etopo1_grayscale_hillshade.tif.zip is valid

Checking Resources/ETOPO_c_geotiff_10800.tif.zip (fbd0478d0974ca433cdd5cb3530d0d48)
Cached file Resources/ETOPO_c_geotiff_10800

In [15]:
for resource in resource_list:
    print "\nDownloading {:s}".format(resource["local_file"])
    download_cached_file(resource["url"], resource["local_file"], resource["md5"], resource["expected_size"] )



Using cached file - Resources/EMAG2_image_V2.tif

Downloaded from Reference/AusMagAll.tiff.zip

Using cached file - Resources/BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF

Using cached file - Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF

Using cached file - Resources/color_etopo1_ice_low.tif.zip

Using cached file - Resources/etopo1_grayscale_hillshade.tif.zip

Using cached file - Resources/ETOPO_c_geotiff_10800.tif.zip

Using cached file - Resources/HYP_50M_SR_W.zip

Using cached file - Resources/OB_50M.zip

Downloaded from Reference/global_age_data.3.6.z.npz

Downloaded from Reference/sec_invariant_strain_0.2.dat.zip

Downloaded from Reference/Earthquake_data.zip

Downloaded from Reference/velocity_AU.nc

Downloaded from Reference/velocity_EU.nc

Downloaded from Reference/velocity_IN.nc

Downloaded from Reference/velocity_NA.nc

Downloaded from Reference/velocity_OK.nc

Downloaded from Reference/velocity_NNR.nc

Downloaded from Reference/velocity_PA.nc

Downloaded from R

In [4]:
# Extract any files that were downloaded as zip archives (keep the originals to avoid re-downloading)

import zipfile
import glob

for zipped in glob.glob("Resources/*.zip"):
    with zipfile.ZipFile(zipped) as zf:
        zf.extractall("Resources")
        print "Unzipped {}".format(zipped)

Unzipped Resources/AusMagAll.tiff.zip
Unzipped Resources/color_etopo1_ice_low.tif.zip
Unzipped Resources/Earthquake_data.zip
Unzipped Resources/etopo1_grayscale_hillshade.tif.zip
Unzipped Resources/ETOPO_c_geotiff_10800.tif.zip
Unzipped Resources/HYP_50M_SR_W.zip
Unzipped Resources/OB_50M.zip
Unzipped Resources/sec_invariant_strain_0.2.dat.zip


In [5]:
# if you add a new url / file to the download list above, add the md5 checksum as well !
# unless it is small enough to have a local copy. I would suggest still providing a url
# in the comments just in case.

md5sum("Resources/etopo1_grayscale_hillshade.tif.zip") 

'f38b8dc6cd971d72e77edaa837d5b85c'

In [6]:
ls -l Resources

total 5323904
-rw-r--r--  1 lmoresi  staff   97372378  1 Mar 13:10 AusMagAll.tiff
-rw-r--r--  1 lmoresi  staff    4895422  1 Mar 12:30 AusMagAll.tiff.zip
-rw-r--r--  1 lmoresi  staff   10714734  1 Mar 11:51 BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF
-rw-r--r--  1 lmoresi  staff   10854863  1 Mar 11:51 BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF
-rw-r--r--  1 lmoresi  staff  134722670  1 Mar 11:51 EMAG2_image_V2.tif
-rw-r--r--  1 lmoresi  staff  466647468  1 Mar 13:10 ETOPO1_Ice_c_geotiff.tif
-rw-rw-r--@ 1 lmoresi  staff  466712272  6 Jun  2011 ETOPO1_Ice_g_geotiff.tif
-rw-r--r--@ 1 lmoresi  staff  266083133 27 Oct 11:36 ETOPO_c_geotiff_10800.tif
-rw-r--r--@ 1 lmoresi  staff  323509293  1 Mar 12:36 ETOPO_c_geotiff_10800.tif.zip
-rw-r--r--  1 lmoresi  staff    5449584  1 Mar 13:10 Earthquake_data.zip
-rw-r--r--  1 lmoresi  staff    6145753  1 Mar 13:10 Earthquakes-2000-2014-5.5+.json
-rw-r--r--  1 lmoresi  staff   12090635  1 Mar 13:10 Earthquakes-AusRegion-2000-2014-4.8-5.5+.json
-